# Recommender System for Board Games from [BoardGameGeek.com](https://boardgamegeek.com/)



1.   Iatrou Manos
2.   Papageorgiou Vasileios
3. Sykianakis Xaralambos




# Dataset Description



*   Games File
*   User Ratings File
*   Mechanics File
*   Themes File



In [1]:
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# List of CSV file paths or filenames
# csv_files = ['games.csv', 'mechanics.csv', 'themes.csv','user_ratings.csv']
folder_path = Path(r'C:\Users\maniat\OneDrive - Lyse AS\Desktop\personal\MSc\Customer Analytics\Recommender Classification\Recommender_Files')
games = pd.read_csv(folder_path / 'games.csv')
mechanics =  pd.read_csv(folder_path / 'mechanics.csv')
themes =  pd.read_csv(folder_path / 'themes.csv')
user_ratings =  pd.read_csv(folder_path /'user_ratings.csv')

In [9]:
columns_to_keep = ['BGGId','GameWeight','MfgPlaytime','NumAlternates','NumExpansions','NumImplementations','Kickstarted',
                   'Cat:Thematic','Cat:Strategy','Cat:War','Cat:Family','Cat:CGS','Cat:Abstract','Cat:Party','Cat:Childrens']

In [10]:
games = games[columns_to_keep]

In [11]:
games['GameWeight'] = games['GameWeight'].apply(lambda x: 'Difficult' if x > 3.5 else ('Medium' if x > 2 else 'Easy'))

C:\Users\maniat\AppData\Local\Temp\ipykernel_8376\852537194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['GameWeight'] = games['GameWeight'].apply(lambda x: 'Difficult' if x > 3.5 else ('Medium' if x > 2 else 'Easy'))


In [12]:
games['MfgPlaytime'] = games['MfgPlaytime'].apply(lambda x: 'Long_Game' if x > 120 else ('Regular_Game' if x > 50 else 'Short_Game'))

C:\Users\maniat\AppData\Local\Temp\ipykernel_8376\1745565743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['MfgPlaytime'] = games['MfgPlaytime'].apply(lambda x: 'Long_Game' if x > 120 else ('Regular_Game' if x > 50 else 'Short_Game'))


In [13]:
merged_df = pd.merge(games, mechanics, on='BGGId')
games = pd.merge(merged_df, themes, on='BGGId')

In [14]:
renaming_dict = {col: col.replace('Theme_', '') if col.startswith('Theme_') 
                 else col.replace('Cat:', '') if col.startswith('Cat:')
                 else col
                 for col in games.columns}
games.rename(columns=renaming_dict,inplace=True)

In [21]:
def get_description(df):
    def categorize(row):
        categories = []
        for col in df.columns[3:]:
            if row[col] > 0:
                category = col
                categories.append(category)
        return ','.join(categories) if categories else 'Other'
    return df.apply(categorize,axis=1)

In [22]:
games['Details'] = get_description(games)

In [25]:
games['Summary'] = games['Details'] + ',' + games['GameWeight'] + ',' + games['MfgPlaytime']

In [28]:
games = games[['BGGId','Summary']]

In [ ]:
user_ratings['sentiment'] = user_ratings['Rating'].apply(lambda x: 0 if x <= 6 else 1)

In [ ]:
user_ratings = user_ratings.drop('Rating', axis=1)

In [ ]:
def downcasting_types(df: pd.DataFrame):
  for column in df.columns:
    colType = df[column].dtype
    if colType == 'float64' :
      df[column] = pd.to_numeric(df[column], downcast='float')
    elif colType == 'int64' :
      df[column] = pd.to_numeric(df[column], downcast='integer')
  return df

In [ ]:
new_gamedf = downcasting_types(games)

In [ ]:
new_userdf = downcasting_types(user_ratings)

In [ ]:
print(new_gamedf.info())
print(new_userdf.info())